# Ushur API Key Renewal
As a best practice, API keys should be rotated frequently. Ushur provides the ability to create keys with expiration dates, as well as revoke active keys. This document will show you how to automatically rotate your keys using the Ushur REST APIs with little to no interruption to your end users.
![Creating Key through the Console](images/create_api.png)

# Key Management Best Practices
Before we get into it, we should ensure that there are baseline best practices incorporated into your key management policies.
1. Even if it's automatic, this procedure should be executed during a maintenance window, or period of low system usage.
2. Rotate Keys about 2 weeks before they expire. This gives you time to deal with any issues with rotation.
3. If you're revoking keys, also do this after you've tested the configuration of your new key.
4. Your new keys should be stored in an (encrypted vault)[https://geekflare.com/secret-management-software/]

# Key Management Process
Rotating your keys will follow the process outlined in the following sequence diagram:
![Sequence Diagram](images/key_mgmt_seq.png)


The diagram assumes that you're also already using a Secrets Vault to store and retrieve your `API_KEY` for use in your integrating application.

#

# Common Imports

In [ ]:
# Common imports
import requests
import pandas as pd
import yaml
configs = {
    "endpoint" : "https://your.ushur.instance/",
    "email": "your_email@example.com",
    "token" : "UR.ushur_token"
}

# Create a New API_KEY
This request will show you how to create a new `API_KEY` using your existing `API_KEY`. The key created will be set to expire in 90 days.

In [ ]:
create_response=requests.post(f"{configs.get('endpoint')}rest/apikey/create",
                json={
                    "userEmail": configs.get('email'),
                    "description": "myNewKey",
                    "expiryDays": "90"
                },
                headers={
                    "content-type": "application/json",
                    "token": configs.get("token")
                } );

Retrieve the key into a variable for storage. Remember **Keep your Keys Safe**

In [ ]:
if create_response.status_code == 200:
    api_key_data = create_response.json().get('data')
else:
    print("Error retrieving list of tokens")
    print(create_response.text)


# List existing keys
To list your existing API_KEY data (not the actual key itself), you will Post the following request to the API `rest/apikey/list` at your Ushur instance endpoint.

In [ ]:
list_response=requests.post(f"{configs.get('endpoint')}rest/apikey/list",
                json={
                    "userEmail": configs.get('email')
                },
                headers={
    "content-type": "application/json",
    "token": configs.get("token")
} );


### Rendering Ouput

In [ ]:
if list_response.status_code == 200:
    j_data = list_response.json().get('data')
    data = pd.DataFrame.from_dict(j_data)
    print(data)
else:
    print("Error retrieving list of tokens")
    print(list_response.text)

# Revoke an API_KEY
To revoke an API_KEY, you will need the `apiKeyId` associated with the `API_KEY` that you want to revoke.


In [ ]:
revoke_response=requests.post(f"{configs.get('endpoint')}rest/apikey/revoke/{api_key_data.get('apiKeyId')}",

                headers={
                    "content-type": "application/json",
                    "token": configs.get("token")
                } );

Print Revocation Output


In [ ]:
if revoke_response.status_code == 200:
    j_data = revoke_response.json()
    print(j_data)
else:
    print("Error retrieving list of tokens")
    print(revoke_response.text)

# Putting it all together
The following Script will take an API key from your configuration store, normally an encrypted vault, create a new key request, then use the new key to revoke access to the old key.

Format for `~/ushur.yml` is
```
ushur:
  apiKey: UR.ushur_apiKey
  apiKeyId: ushurApiKeyId
  email: your_email@example.com
  endpoint: https://ushur_instance/
```

In [79]:

import yaml
import  os
import requests
config_file=os.path.expanduser("~/ushur.yml")

def retrieve_key():
    """
    Retrieve the key from your configuration store
    :return:
    """
    with open(config_file, 'r') as yml:
        return yaml.safe_load(yml)

def create_key(config):
    """
    Use the configurations that  you've loaded to create your new key
    :param config:
    :return:
    """
    response=requests.post(f"{config['ushur']['endpoint']}rest/apikey/create",
                                  json={
                                      "userEmail": config['ushur']['email'],
                                      "description": "myNewKey",
                                      "expiryDays": "90"
                                  },
                                  headers={
                                      "content-type": "application/json",
                                      "token": config['ushur']['apiKey']
                                  } );
    if response.status_code == 200:
        return response.json().get('data')
    else:
        raise Exception(f"Status: {response.status_code} -- Error {response.text}")


def store_key(config, newkey):
    """
    Store the new key into your configuration store. In this example, I'm using an external yaml file.
    :param config:  original configuration object
    :param newkey: your new key
    :return:
    """

    config['ushur']['apiKey']=newkey['apiKey']
    config['ushur']['apiKeyId']=newkey['apiKeyId']
    with open(config_file, 'w') as yml:
        return yaml.dump(config, yml)



def validate_key(config, newkey):
    """
    Use the new key to test your integrations
    :param config: original configuration object
    :param newkey: your new key
    :return:
    """
    response=requests.post(f"{config['ushur']['endpoint']}rest/apikey/list",
                                json={
                                    "userEmail": config['ushur']['email']
                                },
                                headers={
                                    "content-type": "application/json",
                                    "token": newkey["apiKey"]
                                } );
    if response.status_code!=200:
        raise Exception(f"Status: {response.status_code} -- Error: {response.text}")


def revoke_key(config, oldkey):
    """
    Revoke your old key now that your new key is working.
    :param config: configuration object with the newly created key
    :param oldkey: the old key that needs to be revoked.
    :return:
    """
    revoke_response=requests.post(f"{config['ushur']['endpoint']}rest/apikey/revoke/{oldkey.get('apiKeyId')}",

                                  headers={
                                      "content-type": "application/json",
                                      "token": config['ushur']['apiKey']
                                  } );


def rotate_key():
    config = retrieve_key()
    current_key= {
        "apiKeyId": config['ushur']['apiKeyId'],
        "apiKey": config["ushur"]["apiKey"],
    }
    new_key = create_key(config)
    validate_key(config, new_key)
    store_key(config, new_key)
    revoke_key(config, current_key)
    print("Completed")

rotate_key()